In [9]:
import projectpath

import collections
import copy
from importlib import resources

import bokeh.models
import bokeh.plotting as plt
import numpy as np
import pandas as pd
import panel as pn
import param

import files.pw
from kb import kb
from model.core import DbXref, KbEntry, Molecule, Reaction, Pathway

KB = kb.configure_kb()
pn.extension()

# Search

In [10]:
def search_id(term, datasets):
    for dataset in datasets:
        obj = KB.get(dataset, term)
        if obj:
            yield dataset, obj
    

def search_names(term, datasets):
    for dataset in datasets:
        for obj in KB.find(dataset, term, include_aka=True):
            yield dataset, obj


def search_xrefs(term, datasets):
    # 'bar:foo' -> db='bar', id='foo'; 'foo' -> db=None, id='foo'
    parts = [None] + term.split(':', 2)
    for dataset in datasets:
        for obj in KB.xref(dataset, xref_id=parts[-1], xref_db=parts[-2]):
            yield dataset, obj


def search(term, fields, datasets):
    results = collections.defaultdict(set)
    # id
    if 'id' in fields:
        for dataset, obj in search_id(term, datasets):
            results[dataset].add(obj)
    # name/AKA
    if 'name/AKA' in fields:
        for dataset, obj in search_names(term, datasets):
            results[dataset].add(obj)
    # xref
    if 'xref' in fields:
        for dataset, obj in search_xrefs(term, datasets):
            results[dataset].add(obj)
    
    return results

In [11]:
class CardSelector(pn.viewable.Viewer):
    name = param.String(default=None)
    options = param.ClassSelector(default=[], class_=(list, dict))
    value = param.Parameter(default=None)
    
    def __init__(self, name, options, value=None, **card_params):
        super().__init__(name=name, options=options, value=value)
        
        self._selector = pn.widgets.MultiSelect(sizing_mode='stretch_both')
        self.sync_selector_options()
        self.sync_selector_value()

        self._card = pn.Card(
            self._selector,
            header_background='transparent',
            css_classes=['card-select'],
            **card_params,
        )
        self.sync_card_header()
        
        self.param.watch(self.sync_selector_value, ['value'])
        self.param.watch(self.sync_selector_options, ['options'])
        self.param.watch(self.sync_card_header, ['name'])
        self._selector.param.watch(self.sync_value, ['value'])

    def sync_selector_value(self, *_):
        """Updates the underlying selector's value when my value changes."""
        if self.value and self._selector.value != [self.value]:
            self._selector.value = [self.value]
        elif self.value is None and self._selector.value:
            self._selector.value = []

    def sync_selector_options(self, *_):
        """Updates the underlying selector's options when my options change."""
        self._selector.options = self.options
    
    def sync_card_header(self, *_):
        """Updates the underlying Card header when my name changes"""
        if self.name:
            self._card.header = pn.Pane(self.name, style={'font-size': '12px', 'font-weight': 'normal'})
        else:
            self._card.header = None

    def sync_value(self, *_):
        """Updates my (single) value when the underlying selector's value changes."""
        if self._selector.value:
            # _selector allows multi-selection; override to take only a new value
            for option in self._selector.value:
                if option != self.value:
                    self.value = option
                    break

        elif self.value is not None:
            self.value = None

    def __panel__(self):
        return self._card


card = CardSelector(name='Colors', options=['red', 'green', 'blue'], value='green')
pn.Row(card, width=300)

Row(width=300)
    [0] Card(css_classes=['card-select'], header=Markdown, header_background='transparent')
        [0] MultiSelect(options=['red', 'green', 'blue'], sizing_mode='stretch_both', value=['green'])

In [12]:
class SelectorGroup(pn.viewable.Viewer):
    value = param.Parameter(default=None)
    
    def __init__(self, *selectors, value=None, **layout_params):
        super().__init__()  # defer setting value until dependencies are set up
        self._selectors = []
        self._watchers = []
        self._option_map = {}
        self._layout = pn.Column(**layout_params)
        self._ignore_events = set()

        self.set_selectors(selectors)
        self.param.watch(self.sync_selector_values, ['value'])
        if value:
            self.value = value
    
    # Worry about a full list-like API later, if it's warranted.
    def clear(self):
        self.value = None
        for selector, watcher in zip(self._selectors, self._watchers):
            selector.param.unwatch(watcher)
        self._selectors.clear()
        self._watchers.clear()
        self._option_map.clear()
        self._layout.clear()

    def set_selectors(self, selectors):
        """Replace the full set of selectors in this group. This resets any selected value."""
        def iter_options(selector):
            if isinstance(selector.options, dict):
                return selector.options.values()
            else:
                return selector.options
        
        self.clear()
        for selector in selectors:
            for option in iter_options(selector):
                self._option_map[option] = selector 
            self._selectors.append(selector)
            self._watchers.append(selector.param.watch(self.sync_value, ['value']))
            self._layout.append(selector)
        
    @property
    def options(self):
        """Read-only stand-in for selector `options` parameter."""
        return {option: option for option in self._option_map}

    def sync_selector_values(self, *events):
        """Updates the underlying selectors' values when my value changes."""
        event = events[0]
        old_selector = self._option_map.get(event.old)
        new_selector = self._option_map.get(event.new)
        if old_selector and old_selector != new_selector:
            # Clear the previous selection without triggering a resulting update on self.value
            self._ignore_events.add(old_selector)
            old_selector.value = None
            self._ignore_events.remove(old_selector)
        if new_selector and new_selector.value != event.new:
            new_selector.value = event.new

    def sync_value(self, *events):
        """Updates my (single) value when any underlying selector's value changes."""
        if  events[0].obj in self._ignore_events:
            return

        if self.value != events[0].new:
            self.value = events[0].new

    def __panel__(self):
        return self._layout
    
SelectorGroup(
    CardSelector(name='color', options=['red', 'green', 'blue'], background='#fffff0'),
    CardSelector(name='shape', options=['round', 'square'], background='#fffff0'),
    value='round',
    background='#fff8f8',
)

SelectorGroup(name='SelectorGroup00230', value='round')

In [13]:
searchbox = pn.widgets.TextInput(name='🔍', sizing_mode='stretch_width')
working_set = SelectorGroup()

# @param.depends(searchbox.value, watch=True)
def update_results(*events):
    working_set.clear()
    term = searchbox.value
    if term:
        selectors = []
        for ds, objs in search(term, ['id', 'name/AKA', 'xref'], list(KB.schema.values())).items():
            selectors.append(CardSelector(name=ds.collection, options={obj.name: obj for obj in objs}, width=290))
        print(f'{len(selectors)} selectors')
        working_set.set_selectors(selectors)

watcher = searchbox.param.watch(update_results, ['value'])
sidebar = pn.Column(
    searchbox,
    pn.Row(working_set),
    width=300,
)

sidebar

Column(width=300)
    [0] TextInput(name='🔍', sizing_mode='stretch_width')
    [1] Row
        [0] Column()

# Edit

In [14]:
pn.Row(
    pn.widgets.Button(name='⧉', height=36, width=36), # two squares u29c9
    pn.widgets.Button(name='⇢', height=36, width=36), # dashed arrow u21e2
    pn.widgets.Button(name='✗', height=36, width=36), # ballot x u2717
    pn.widgets.Button(name='✓', height=36, width=36), # check u2713
    pn.widgets.Button(name='⟲', height=36, width=36), # anticlockwise gapped arrow u27f2
    pn.Spacer(width=18),
    pn.widgets.Button(name='→', height=36, width=36), # plain arrow u2192
    pn.widgets.Button(name='✘', height=36, width=36), # heavy ballot x u2718
    pn.widgets.Button(name='💾', height=36, width=36), # save u1f4be
    pn.widgets.Button(name='🗑', height=36, width=36), # wastebasket u1f5d1
    pn.widgets.Button(name='⎌', height=36, width=36), # undo u238c
)

Row
    [0] Button(height=36, name='⧉', sizing_mode='fixed', width=36)
    [1] Button(height=36, name='⇢', sizing_mode='fixed', width=36)
    [2] Button(height=36, name='✗', sizing_mode='fixed', width=36)
    [3] Button(height=36, name='✓', sizing_mode='fixed', width=36)
    [4] Button(height=36, name='⟲', sizing_mode='fixed', width=36)
    [5] Spacer(width=18)
    [6] Button(height=36, name='→', sizing_mode='fixed', width=36)
    [7] Button(height=36, name='✘', sizing_mode='fixed', width=36)
    [8] Button(height=36, name='💾', sizing_mode='fixed', width=36)
    [9] Button(height=36, name='🗑', sizing_mode='fixed', width=36)
    [10] Button(height=36, name='⎌', sizing_mode='fixed', width=36)

In [15]:
class KbEntryEditor:
    def __init__(self, dataset, entry):
        self._entry = entry
        self._source_dataset = dataset
        self._source_id = entry.id
        self._dest_dataset = None
        
        # Set up editor fields
        self.dataset = pn.pane.Markdown(width=100, align='center', style={'text-align': 'right'})
        self.id = pn.widgets.TextInput(disabled=True)
        self.name = pn.widgets.TextInput(name='name')
        self.shorthand = pn.widgets.TextInput(name='shorthand', width=100)
        self.description = pn.widgets.TextAreaInput(name='description', height=100)
        self.aka = pn.widgets.TextAreaInput(name='aka', height=120)
        self.xrefs = pn.widgets.TextAreaInput(name='xrefs', height=120)
        
        # Set up action buttons
        self.clone_button = pn.widgets.Button(name='⧉', height=36, width=36, align='end')
        self.clone_button.on_click(self.clone_entry)
        self.move_button = pn.widgets.Button(name='⇢', height=36, width=36, align='end')
        self.move_button.on_click(self.move_entry)
        self.delete_button = pn.widgets.Button(name='✗', height=36, width=36, align='end')
        self.delete_button.on_click(self.delete_entry)
        self.confirm_button = pn.widgets.Button(name='✓', height=36, width=36, align='end')
        self.confirm_button.on_click(self.confirm)
        
        # Start in the base state
        self.reset()

    def layout(self):
        return pn.Column(
            pn.Row(self.dataset, self.id, pn.Row(self.clone_button, self.move_button, self.delete_button, align='end')),
            pn.Row(self.name, self.shorthand),
            self._main_attributes(),
            pn.Row(self.aka, self.xrefs),
            self.description,
            self.confirm_button,
        )
    
    def _main_attributes(self):
        return pn.Row()
    
    def sync_fields(self):
        self.dataset.object = self._source_dataset.collection + ' :'
        self.id.value = str(self._entry.id)
        self.id.disabled = True
        
        self.name.value = self._entry.name
        self.shorthand.value = self._entry.shorthand or ''
        self.description.value = self._entry.description or ''
        self.aka.value = '\n'.join(self._entry.aka or [])
        self.xrefs.value = '\n'.join(str(xref) for xref in (self._entry.xrefs or []))
        
    def sync_entry(self):
        # TODO: validate everything
        self._entry.name = self.name.value
        self._entry.shorthand = self.shorthand.value or None
        self._entry.description = self.description.value or None
        
        aka = []
        for line in self.aka.value.split('\n'):
            line = line.strip()
            if line:
                aka.append(line)
        self._entry.aka = aka or None
        
        xrefs = []
        for line in self.xrefs.value.split('\n'):
            line = line.strip()
            if line:
                xrefs.append(DbXref.from_str(line))
        self._entry.xrefs = xrefs or None

    def reset(self):
        """Restore field values to reflect the current persistent state of the entry."""
        self.sync_fields()
        
        editable = self._source_dataset.db == 'kb'
        for button in (self.clone_button, self.move_button, self.delete_button, self.confirm_button):
            button.button_type = 'default'
            button.disabled = not editable
        self.clone_button.disabled = False
        
    def clone_destination(self):
        return None

    def clone_entry(self, event):
        self._dest_dataset = self.clone_destination()
        if self._dest_dataset:
            self.dataset.object = self._dest_dataset.collection + ' :'
            self.id.value = ''
            self.id.disabled = False
            self.confirm_button.disabled = False
        
        # Cross-reference to the original, but only from a reference source
        if self._source_dataset.db == 'ref':
            self.xrefs.value = f'{self._source_dataset.collection}:{self._source_id}\n' + self.xrefs.value
            
        self.clone_button.button_type='primary'
        self.move_button.disabled = True
        self.delete_button.disabled = True
        self.confirm_button.disabled = False
        
    def move_entry(self, event):
        self.id.disabled = False
        self.move_button.button_type='primary'
        self.clone_button.disabled = True
        self.delete_button.disabled = True
        self.confirm_button.disabled = False
        
    def delete_entry(self, event):
        self._delete_pending = True
        self.delete_button.button_type='primary'
        self.clone_button.disabled = True
        self.move_button.disabled = True
        self.confirm_button.disabled = False
    
    def confirm(self, event):
        if self._dest_dataset:
            self._entry = copy.deepcopy(self._entry)
            self._entry._id = self.id.value        
        self.sync_entry()

        self._source_id = self._entry.id
        if self._dest_dataset:
            self._source_dataset = self._dest_dataset
            self._dest_dataset = None
        self.reset()


class CompoundEditor(KbEntryEditor):
    def __init__(self, dataset, entry):
        self.formula = pn.widgets.TextInput(name='formula', width=200)
        self.mass = pn.widgets.FloatInput(name='mass', width=100)
        self.charge = pn.widgets.IntInput(name='charge', width=100)
        self.inchi = pn.widgets.TextInput(name='InChi')
        super().__init__(dataset, entry)

    def _main_attributes(self):
        return pn.Column(
            pn.Row(self.formula, self.mass, self.charge),
            self.inchi,
        )

    def sync_fields(self):
        super().sync_fields()
        self.formula.value = self._entry.formula
        self.charge.value = self._entry.charge
        self.mass.value = self._entry.mass
        self.inchi.value = self._entry.inchi
        
    def sync_entry(self):
        super().sync_entry()
        self._entry.formula = self.formula.value or None
        self._entry.charge = self.charge.value
        self._entry.mass = self.mass.value
        self._entry.inchi = self.inchi.value
        
    def clone_destination(self):
        return KB.compounds


class ReactionEditor(KbEntryEditor):
    def __init__(self, dataset, entry):
        self.reactants = pn.Card(header=pn.pane.Markdown('stoichiometry'), header_background='transparent')
        for reactant, count in entry.stoichiometry.items():
            self.reactants.append(pn.Row(pn.widgets.IntInput(width=60), pn.widgets.TextInput(), margin=(0, 30)))
        self.catalyst = pn.widgets.TextInput(name='catalyst')
        self.reversible = pn.widgets.Checkbox(name='reversible', align='center')
        super().__init__(dataset, entry)

    def _main_attributes(self):
        return pn.Column(
            self.reactants,
            pn.Row(self.catalyst, self.reversible),
        )

    def sync_fields(self):
        super().sync_fields()
        for (count_widget, reactant_widget), (reactant, count) in zip(self.reactants, self._entry.stoichiometry.items()):
            reactant_widget.value = reactant.id
            count_widget.value = count
        if self._entry.catalyst:
            self.catalyst.value = self._entry.catalyst.id
        else:
            self.catalyst.value = None
        self.reversible.value = self._entry.reversible
        
    def sync_entry(self):
        super().sync_entry()
        stoichiometry = {}
        for (count_widget, reactant_widget), (reactant, count) in zip(self.reactants, self._entry.stoichiometry.items()):
            reactant._id = reactant_widget.value
            stoichiometry[reactant] = count_widget.value
        self._entry.stoichiometry = stoichiometry
        if self._entry.catalyst:
            self._entry.catalyst._id = self.catalyst.value
        self._entry.reversible = self.reversible.value
        
    def clone_destination(self):
        return KB.reactions


# Pull it together

In [16]:
searchbox = pn.widgets.TextInput(name='🔍', sizing_mode='stretch_width')
working_set = SelectorGroup()

def update_results(*events):
    working_set.clear()
    term = searchbox.value
    if term:
        selectors = []
        for ds, objs in search(term, ['id', 'name/AKA', 'xref'], list(KB.schema.values())).items():
            selectors.append(CardSelector(name=ds.collection, options={obj.name: obj for obj in objs}, width=290))
        # print(f'{len(selectors)} selectors')
        working_set.set_selectors(selectors)

searcher = searchbox.param.watch(update_results, ['value'])

sidebar = pn.Column(
    searchbox,
    pn.Row(working_set),
    width=300
)
editor_pane = pn.Column(sizing_mode='stretch_both')
layout = pn.Row(
    sidebar,
    editor_pane,
)

def build_editor(dataset, entry):
    editor_class = {
        Molecule: CompoundEditor,
        Reaction: ReactionEditor,
    }
    return editor_class.get(dataset.content_type, KbEntryEditor)(dataset, entry)
    
def launch_editor(*events):
    editor_pane.clear()
    entry = working_set.value
    if entry:
        # Ok, we need to support this more directly...
        dataset = KB.schema[working_set._option_map[entry].name]
        editor_pane.append(build_editor(dataset, entry).layout())
        
launcher = working_set.param.watch(launch_editor, ['value'])

layout

Row
    [0] Column(width=300)
        [0] TextInput(name='🔍', sizing_mode='stretch_width')
        [1] Row
            [0] Column()
    [1] Column(sizing_mode='stretch_both')

## Alt version of editors

In [11]:
class KbEntryEditor:
    def __init__(self, dataset, entry):
        self.dataset = dataset
        self.entry = entry
        self.delete_id = None
        
        # Set up editor fields
        self.ds = pn.pane.Markdown(width=100, align='center', style={'text-align': 'right'})
        self.id = pn.widgets.TextInput(disabled=True)
        self.name = pn.widgets.TextInput(name='name')
        self.shorthand = pn.widgets.TextInput(name='shorthand', width=100)
        self.description = pn.widgets.TextAreaInput(name='description', height=100)
        self.aka = pn.widgets.TextAreaInput(name='aka', height=120)
        self.xrefs = pn.widgets.TextAreaInput(name='xrefs', height=120)
        
        # Set up action buttons
        self.clone_button = pn.widgets.Button(name='⧉', height=36, width=36, align='end')
        self.clone_button.on_click(self.clone_entry)
        self.move_button = pn.widgets.Button(name='⇢', height=36, width=36, align='end')
        self.move_button.on_click(self.move_entry)
        # Put this back when I'm happier with the danger level
        # self.delete_button = pn.widgets.Button(name='✗', height=36, width=36, align='end')
        # self.delete_button.on_click(self.delete_entry)
        self.confirm_button = pn.widgets.Button(name='✓', height=36, width=36, align='end')
        self.confirm_button.on_click(self.confirm)
        
        # Start in the base state
        self.sync_fields()
        self.update_buttons()

    def layout(self):
        return pn.Column(
            pn.Row(self.ds, self.id, pn.Row(self.clone_button, self.move_button, align='end')),
            pn.Row(self.name, self.shorthand),
            self._main_attributes(),
            pn.Row(self.aka, self.xrefs),
            self.description,
            self.confirm_button,
        )
    
    def _main_attributes(self):
        """Overridden in subclasses to provide additional attributes to the layout."""
        return pn.Row()
    
    def sync_fields(self):
        self.ds.object = self.dataset.collection + ' :'
        self.id.value = str(self.entry.id)
        self.id.disabled = True
        
        self.name.value = self.entry.name
        self.shorthand.value = self.entry.shorthand or ''
        self.description.value = self.entry.description or ''
        self.aka.value = '\n'.join(self.entry.aka or [])
        self.xrefs.value = '\n'.join(str(xref) for xref in (self.entry.xrefs or []))
        
    def sync_entry(self):
        # TODO: validate everything
        self.entry._id = self.id.value
        self.entry.name = self.name.value
        self.entry.shorthand = self.shorthand.value or None
        self.entry.description = self.description.value or None
        
        aka = []
        for line in self.aka.value.split('\n'):
            line = line.strip()
            if line:
                aka.append(line)
        self.entry.aka = aka or None
        
        xrefs = []
        for line in self.xrefs.value.split('\n'):
            line = line.strip()
            if line:
                xrefs.append(DbXref.from_str(line))
        self.entry.xrefs = xrefs or None

    def update_buttons(self):
        # Not currently valid to do anything with root KbEntry itself
        for button in (self.clone_button, self.move_button, self.confirm_button):
            button.button_type = 'default'
            button.disabled = True
        
    def _make_entry_copy(self):
        # Not currently valid to do anything with root KbEntry itself...
        return None, None

    def clone_entry(self, event):
        dataset, entry = self._make_entry_copy()
        if entry:
            entry._id = ''
            # Cross-reference to the original, but only from a reference source
            if self.dataset.db == 'ref':
                entry.xrefs.add(DbXref(self.dataset.collection, self.entry.id))

            self.dataset = dataset
            self.entry = entry
            self.sync_fields()
            
            self.id.disabled = False
            self.clone_button.button_type='primary'
            self.move_button.disabled = True
            self.confirm_button.disabled = False
        
    def move_entry(self, event):
        self.delete_id = self.entry.id
        self.id.disabled = False
        self.move_button.button_type='primary'
        self.clone_button.disabled = True
        self.confirm_button.disabled = False
        
    def delete_entry(self, event):
        self.delete_id = self.entry.id
        # self.delete_button.button_type='primary'
        self.clone_button.disabled = True
        self.move_button.disabled = True
        self.confirm_button.disabled = False
    
    def confirm(self, event):
        self.sync_entry()
        if self.delete_id:
            print('***DELETE***')
            self.delete_id = None
        print('***SAVE***')

        self.update_buttons()


class CompoundEditor(KbEntryEditor):
    def __init__(self, dataset, entry):
        self.formula = pn.widgets.TextInput(name='formula', width=200)
        self.mass = pn.widgets.FloatInput(name='mass', width=100)
        self.charge = pn.widgets.IntInput(name='charge', width=100)
        self.inchi = pn.widgets.TextInput(name='InChi')
        super().__init__(dataset, entry)

    def _main_attributes(self):
        return pn.Column(
            pn.Row(self.formula, self.mass, self.charge),
            self.inchi,
        )

    def sync_fields(self):
        super().sync_fields()
        self.formula.value = self.entry.formula
        self.charge.value = self.entry.charge
        self.mass.value = self.entry.mass
        self.inchi.value = self.entry.inchi
        
    def sync_entry(self):
        super().sync_entry()
        self.entry.formula = self.formula.value or None
        self.entry.charge = self.charge.value
        self.entry.mass = self.mass.value
        self.entry.inchi = self.inchi.value

    def update_buttons(self):
        super().update_buttons()
        self.clone_button.disabled = False
        self.move_button.disabled = self.dataset.db != 'kb'
        self.confirm_button.disabled = self.dataset.db != 'kb'
        
    def _make_entry_copy(self):
        return KB.compounds, copy.deepcopy(self.entry)


class ReactionEditor(KbEntryEditor):
    def __init__(self, dataset, entry):
        self.reactants = pn.Card(header=pn.pane.Markdown('stoichiometry'), header_background='transparent')
        for reactant, count in entry.stoichiometry.items():
            self.reactants.append(pn.Row(pn.widgets.IntInput(width=60), pn.widgets.TextInput(), margin=(0, 30)))
        self.catalyst = pn.widgets.TextInput(name='catalyst')
        self.reversible = pn.widgets.Checkbox(name='reversible', align='center')
        super().__init__(dataset, entry)

    def _main_attributes(self):
        return pn.Column(
            self.reactants,
            pn.Row(self.catalyst, self.reversible),
        )

    def sync_fields(self):
        super().sync_fields()
        for (count_widget, reactant_widget), (reactant, count) in zip(self.reactants, self.entry.stoichiometry.items()):
            reactant_widget.value = reactant.id
            count_widget.value = count
        if self.entry.catalyst:
            self.catalyst.value = self.entry.catalyst.id
        else:
            self.catalyst.value = None
        self.reversible.value = self.entry.reversible
        
    def sync_entry(self):
        super().sync_entry()
        stoichiometry = {}
        for (count_widget, reactant_widget), (reactant, count) in zip(self.reactants, self.entry.stoichiometry.items()):
            reactant._id = reactant_widget.value
            stoichiometry[reactant] = count_widget.value
        self.entry.stoichiometry = stoichiometry
        if self.entry.catalyst:
            self.entry.catalyst._id = self.catalyst.value
        self.entry.reversible = self.reversible.value
 
    def update_buttons(self):
        super().update_buttons()
        self.clone_button.disabled = False
        self.move_button.disabled = self.dataset.db != 'kb'
        self.confirm_button.disabled = self.dataset.db != 'kb'
       
    def _make_entry_copy(self):
        stoichiometry = {}
        for reactant, count in self.entry.stoichiometry.items():
            # TODO: we need to know that e.g. RHEA references CHEBI and KB.reactions references KB.compounds
            xrefs = KB.xref(KB.compounds, xref_id=reactant.id)
            if len(xrefs) == 1:
                kb_reactant = xrefs[0]
                if kb_reactant.canonical_form:
                    kb_reactant = KB.get(KB.compounds, kb_reactant.canonical_form.parent_id)
                stoichiometry[kb_reactant] = count
            else:
                return None, None
        
        # Made it this far -- we have valid KB versions of all reactants
        new_reaction = copy.deepcopy(self.entry)
        new_reaction.stoichiometry = stoichiometry        
        return KB.reactions, new_reaction


In [12]:
searchbox = pn.widgets.TextInput(name='🔍', sizing_mode='stretch_width')
working_set = SelectorGroup()

def update_results(*events):
    working_set.clear()
    term = searchbox.value
    if term:
        selectors = []
        for ds, objs in search(term, ['id', 'name/AKA', 'xref'], list(KB.schema.values())).items():
            selectors.append(CardSelector(name=ds.collection, options={obj.name: obj for obj in objs}, width=290))
        # print(f'{len(selectors)} selectors')
        working_set.set_selectors(selectors)

searcher = searchbox.param.watch(update_results, ['value'])

sidebar = pn.Column(
    searchbox,
    pn.Row(working_set),
    width=300
)
editor_pane = pn.Column(sizing_mode='stretch_both')
layout = pn.Row(
    sidebar,
    editor_pane,
)

def build_editor(dataset, entry):
    editor_class = {
        Molecule: CompoundEditor,
        Reaction: ReactionEditor,
    }
    return editor_class.get(dataset.content_type, KbEntryEditor)(dataset, entry)
    
def launch_editor(*events):
    editor_pane.clear()
    entry = working_set.value
    if entry:
        # Ok, we need to support this more directly...
        dataset = KB.schema[working_set._option_map[entry].name]
        editor_pane.append(build_editor(dataset, entry).layout())
        
launcher = working_set.param.watch(launch_editor, ['value'])

layout

Row
    [0] Column(width=300)
        [0] TextInput(name='🔍', sizing_mode='stretch_width')
        [1] Row
            [0] Column()
    [1] Column(sizing_mode='stretch_both')



eldolan@uga.edu
Mariel.pfeifer@uga.edu
